In [1]:
import spacy
nlp = spacy.load('en_core_web_lg', disable=["parser", "tagger", "ner"])

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)

In [396]:
fun_fact_df = pd.read_csv('../data/fun_fact_title.csv')

/Users/danny/Documents/College-CORNELL/04 Senior-MEng/Spring 2019/INFO4300 - Language and Information/final project/fun-fax/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,8,10,11,13,14,15,16,17,19,20,21,22,23,24,25,26,28,29,30,31,33,37,38,43,44,47,48,49,50,51,52,54,55,57,58,60,61,62,64,65,71,75,87,89,91,92,93,94,97,101,104,105,106,107,110,111,112,113,114,115,118,119,120,121,122,129,133,135,137,138,139,140,142,143,148,152,155,174,176,178,179,180,181,187,191,192,193,194,197,199,201,202,203,209) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [389]:
required_columns = ['title', 'subreddit', 'permalink']
fun_fact_df = fun_fact_df.dropna(axis='rows', subset=required_columns).reset_index(drop=True)

In [390]:
fun_fact_df = fun_fact_df.drop_duplicates(subset=['title']).reset_index(drop=True)

In [391]:
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy

In [394]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=2, max_df=.8, ngram_range=(2,2))
titles = fun_fact_df['title']
fun_fact_tfidf = vectorizer.fit_transform(titles)

In [393]:
fun_fact_tfidf

<316728x333377 sparse matrix of type '<class 'numpy.float64'>'
	with 1521316 stored elements in Compressed Sparse Row format>

In [367]:
assert "fun" not in vectorizer.get_feature_names()
# assert "donald trump" in vectorizer.get_feature_names()

In [368]:
len(vectorizer.get_feature_names())

24115

In [369]:
len(fun_fact_df['title'])

33268

In [370]:
features = vectorizer.get_feature_names()
weighted_embedding = np.zeros((len(titles), 300))
for row, col in zip(*fun_fact_tfidf.nonzero()):
    feature_vector = np.average([nlp.vocab[w].vector for w in features[col].split()], axis=0)
    weighted_embedding[row] += feature_vector * fun_fact_tfidf[row,col]

In [371]:
avg_weighted = np.array(weighted_embedding) # / (np.sum(fun_fact_tfidf, axis=1) + .00001))

In [372]:
np.sum(fun_fact_tfidf > 0, axis=1) # shows number of tfidf words per doc

matrix([[1],
        [0],
        [0],
        ...,
        [2],
        [1],
        [1]])

In [373]:
print(np.linalg.norm(avg_weighted[21796]))
print(np.linalg.norm(avg_weighted[753]))

9.70781743279948
7.638313249979812


In [387]:
query = "bitcoin food"
query_tfidf = vectorizer.transform([query])
query_weighted = np.zeros(300)
for row, col in zip(*query_tfidf.nonzero()):
    feature_vector = np.average([nlp.vocab[w].vector for w in features[col].split()], axis=0)
    query_weighted += feature_vector * query_tfidf[row,col]
rankings = avg_weighted.dot(query_weighted)
fun_fact_df["title"][np.argsort(-rankings)]

0        Fun fact about the Vatican                                                                                                                                                                                                                      
22187    Fun Fact: From July 25th to September 25th, the Cleveland Indians will have a Home-Road game ratio of 39-19.                                                                                                                                    
22186    give me a random topic and i’ll give u a fun fact about it if i know one                                                                                                                                                                        
22185    TPangolin Co. Ltd. on Twitter: "Here's a fun fact: Vietnam (30 mins) takes twice as long to process as the Boers in-game. Australia takes nearly 4 times as long as Vietnam."                                                                   


In [307]:
query_tfidf.data

array([0.60747938, 0.79433544])

In [359]:
rankings[np.argsort(-rankings)][:5]

array([50.12093631, 48.35141373, 47.01346365, 46.8237791 , 46.6925671 ])

In [309]:
np.dot(avg_weighted[753], query_weighted)

29.999835174077806

In [310]:
len(titles) - len(set(titles))

0

In [378]:
fun_fact_tfidf[11750]

<1x24115 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [385]:
features[4012]

'clean sheet'